## 1) Load data

In [ ]:
in_path = '.'


detection_subdirectory = '.'
# file_pattern = 'RadialSymmetry_results*_ch2*.csv'
# file_pattern = 'fig2.csv'
file_pattern = 'merge_fixed_filenames_goodonly.csv'

# pixel_columns = [f"{d}_2" for d in "zyx"]
pixel_columns = [f"gauss_fit_mu_{d}" for d in "zyx"]
# pixel_columns = list('zyx')

unit_columns = [f"{d}_micron_shift_corrected" for d in "zyx"]
needs_pixel_size = False


channel_column = 'channel'
image_file_column = 'image_file'

split_channel_filename = False

In [ ]:
from pathlib import Path

in_files = sorted((Path(in_path) / detection_subdirectory).glob(file_pattern))
in_files

In [ ]:
import pandas as pd
from calmutils.misc.file_utils import get_common_subpath

df = pd.concat([pd.read_csv(in_file) for in_file in in_files]).reset_index(drop=True)

if split_channel_filename:
    df[image_file_column] = df['img'].str.rsplit("_", n=1, expand=True)[0]

if needs_pixel_size:

    from calmutils.imageio.nd2_helpers import get_pixel_size

    dfis = []

    for file, dfi in df.groupby(image_file_column):

        # get file prefixes from remote paths in table and local mount (in_path)
        # NOTE: we assume the paths share at least some common subpath
        _, (prefix_remote, prefix_local), _ = get_common_subpath(file, in_path)

        pixel_size = get_pixel_size(file.replace(prefix_remote, prefix_local, 1))
        dfi[unit_columns] = dfi[pixel_columns] * pixel_size
        dfis.append(dfi)

    df = pd.concat(dfis)


In [ ]:
df

## 2) Check subpixel offsets

In [ ]:
d_subpixel = np.linalg.norm(df[pixel_columns].values - df[list('zyx')].values, axis=1)
plt.hist(d_subpixel[d_subpixel<1], bins=100);


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np


fig, axs = plt.subplots(ncols=3, figsize=(12,3))

for dim, ax in zip(pixel_columns, axs):

    a = df[dim] - np.round(df[dim])

    # sns.histplot(df, x=a, ax=ax, hue=df[channel_column].astype(str), element='poly', stat='probability', common_norm=False)
    sns.histplot(df, x=a, ax=ax, element='poly', stat='probability', common_norm=False)
    # ax.hist(a, bins=20)
    ax.set_title(dim)

fig.tight_layout()



In [ ]:
df_plot = df.melt(id_vars='channel', value_vars=pixel_columns, var_name='dimension', value_name='location')
df_plot['subpixel_offset'] = df_plot['location'] - df_plot['location'].round()
df_plot

grid = sns.FacetGrid(df_plot, col='dimension', hue='channel', height=3.5)
grid.map(sns.histplot, 'subpixel_offset', element='step', stat='density', alpha=0.1, common_norm=False, bins=31)
plt.legend()

In [ ]:
grid.savefig("/home/david/Desktop/gs651_subpixel_offset_rsfish.pdf")

## 3) Distance between matched coordinates (remaining shift)

In [ ]:
from calmutils.localization.metrics import get_coord_distance_matrix
from scipy.optimize import linear_sum_assignment

max_dist = 0.25

ds_collected = []
ks = []

for k, dfi in df.groupby(image_file_column):

    if len(dfi[channel_column].unique()) != 2:
        continue

    (_, dfi_ch1), (_, dfi_ch2) = dfi.groupby(channel_column)
    coords_ch1 = dfi_ch1[unit_columns].values
    coords_ch2 = dfi_ch2[unit_columns].values

    d = get_coord_distance_matrix(coords_ch1, coords_ch2)
    d[d>max_dist] = max_dist * 9000

    # get optimal matching
    ci, ri = linear_sum_assignment(d)

    coords_ch1_matched = coords_ch1[ci[d[ci, ri] < max_dist]]
    coords_ch2_matched = coords_ch2[ri[d[ci, ri] < max_dist]]

    ds = coords_ch1_matched - coords_ch2_matched

    ds_collected.append(ds)
    ks.extend([k] * len(ds))

In [ ]:
from calmutils.descriptors import match_descriptors_kd
from calmutils.descriptors import descriptor_local_qr

descriptor_match_ratio = 2
max_dist = 0.25

ds_collected = []
ks = []

for k, dfi in df.groupby(image_file_column):

    if len(dfi[channel_column].unique()) != 2:
        continue

    (_, dfi_ch1), (_, dfi_ch2) = dfi.groupby(channel_column)
    coords_ch1 = dfi_ch1[unit_columns].values
    coords_ch2 = dfi_ch2[unit_columns].values

    desc_ch1, idx_ch1 = descriptor_local_qr(coords_ch1, redundancy=2, scale_invariant=True, progress_bar=False)
    desc_ch2, idx_ch2 = descriptor_local_qr(coords_ch2, redundancy=2, scale_invariant=True, progress_bar=False)

    matches = match_descriptors_kd(desc_ch1, desc_ch2, max_ratio=1/descriptor_match_ratio)

    coords_ch1_matched = coords_ch1[idx_ch1[matches.T[0]]]
    coords_ch2_matched = coords_ch2[idx_ch2[matches.T[1]]]

    ds = coords_ch1_matched - coords_ch2_matched

    # discard matches above max distance
    sel = np.linalg.norm(ds, axis=1) <= max_dist
    ds = ds[sel]
    coords_ch1_matched = coords_ch1_matched[sel]
    coords_ch2_matched = coords_ch2_matched[sel]

    ds_collected.append(ds)
    ks.extend([k] * len(ds))

In [ ]:
ds = np.concat(ds_collected)


fig, axs = plt.subplots(ncols=3, figsize=(12,3))

for i, (dim, ax) in enumerate(zip(unit_columns, axs)):

    # sns.histplot(x=ds[:,i], ax=ax, element='poly', stat='density', common_norm=False, hue=ks, legend=False)
    sns.histplot(x=ds[:,i], ax=ax, element='step', stat='density', common_norm=False, legend=False, alpha=0.1)
    # ax.hist(a, bins=20)
    ax.set_title(dim)
    ax.set_xlim((-0.25, 0.25))

sns.despine()
print(f"nr. of matched points: {len(ds)}")
print(f"mean shift: {np.round(ds.mean(axis=0), 4)}")
print(f"shift std. dev.: {np.round(ds.std(axis=0), 4)}")

print(f"shift magnitude median: {np.round(np.median(np.linalg.norm(ds, axis=1)), 4)}")

np.round(np.cov(ds.T), 4)

# plt.savefig("/home/david/Documents/PromoterEnhancer_Revision/PowerFigure/corrected_shifts.pdf")

## Various

In [ ]:
# yaw (angle in xy)
sns.histplot(x=np.atan2(*ds.T[[1,2]]), bins=30, element='step')

# pitch (angle between z and xy projection (magnitude))
# NOTE: result in -pi/2, pi/2, so for visualization purposes we multiply with 2
sns.histplot(x=2 * np.atan2(ds[:,0], np.linalg.norm(ds[:, [1,2]], axis=1)), bins=30, element='step')

plt.ylim((0, 100))

In [ ]:
import nd2
import napari

example_file = next(iter(dfi["image_file"]))

data = nd2.imread(example_file)
data.shape

viewer = napari.view_image(data[:,1], colormap='cyan', blending='additive')
viewer.add_points(coords_ch1_matched, border_color='cyan', face_color='#0000', out_of_slice_display=True)

viewer.add_image(data[:,2], colormap='magenta', blending='additive')
viewer.add_points(coords_ch2_matched, border_color='magenta', face_color='#0000', out_of_slice_display=True)

In [ ]:
import nd2

example_file = "/Volumes/agl_data/NanoFISH/Gabi/GS651_Nanog_2-color_2nM/raw/fov_001.nd2"
example_file = "/Volumes/agl_data/NanoFISH/Gabi/GS651_Nanog_2-color_2nM/raw/Point0000_Point0000_Channel405 CSU-W1,561 CSU-W1,640 CSU-W1 _Seq0000.nd2"

with nd2.ND2File(example_file) as reader:
    m = reader.experiment

from pprint import pprint
pprint(m)